<a href="https://colab.research.google.com/github/barassah/FakeReviews/blob/main/Using_pseudo_labeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
yacharki_yelp_reviews_for_sa_finegrained_5_classes_csv_path = kagglehub.dataset_download('yacharki/yelp-reviews-for-sa-finegrained-5-classes-csv')

print('Data source import complete.')


100%|██████████| 191M/191M [00:10<00:00, 18.5MB/s]

Extracting files...


Data source import complete.


In [6]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [3]:
#Loading important Libraries
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
#from textblob import TextBlob

In [7]:
#loading the data file
dataset=pd.read_csv("train.csv", encoding='ISO-8859-1')
print(dataset)


        class_index                                        review_text
0                 5  dr. goldberg offers everything i look for in a...
1                 2  Unfortunately, the frustration of being Dr. Go...
2                 4  Been going to Dr. Goldberg for over 10 years. ...
3                 4  Got a letter in the mail last week that said D...
4                 1  I don't know what Dr. Goldberg was like before...
...             ...                                                ...
649995            5  I had a sprinkler that was gushing... pipe bro...
649996            1  Phone calls always go to voicemail and message...
649997            1  Looks like all of the good reviews have gone t...
649998            5  I was able to once again rely on Yelp to provi...
649999            1  I have been using this company for 11 months. ...

[650000 rows x 2 columns]


In [8]:
# Getting the shape of the dataset
dataset.shape

(650000, 2)

In [9]:
#Define the columns
dataset.columns

Index(['class_index', 'review_text'], dtype='object')

In [10]:
dataset = dataset[['review_text']]  # Keep only text column
print(dataset)

                                              review_text
0       dr. goldberg offers everything i look for in a...
1       Unfortunately, the frustration of being Dr. Go...
2       Been going to Dr. Goldberg for over 10 years. ...
3       Got a letter in the mail last week that said D...
4       I don't know what Dr. Goldberg was like before...
...                                                   ...
649995  I had a sprinkler that was gushing... pipe bro...
649996  Phone calls always go to voicemail and message...
649997  Looks like all of the good reviews have gone t...
649998  I was able to once again rely on Yelp to provi...
649999  I have been using this company for 11 months. ...

[650000 rows x 1 columns]


In [11]:
# Download NLTK stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Text cleaning function
def clean_text(text):
    text = text.lower()
    # Modified regex to preserve words like "can't" and "don't"
    text = re.sub(r'[^\w\s]+', ' ', text) # remove punctuation but keep letters, numbers, and spaces
    text = ' '.join([word for word in text.split() if word not in stopwords.words('english')])
    return textn,hj

# Apply text cleaning
dataset['review_text'] = dataset['review_text'].apply(clean_text)


In [1]:
# Convert text into numerical features
vectorizer = TfidfVectorizer(max_features=5000)
X_textual = vectorizer.fit_transform(dataset['review_text'])

NameError: name 'TfidfVectorizer' is not defined

In [ ]:
# Cluster into 2 groups (fake vs genuine)
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
pseudo_labels = kmeans.fit_predict(X_textual)

# Add pseudo-labels to the dataset
dataset['label'] = pseudo_labels
print(dataset)

NameError: name 'KMeans' is not defined

In [ ]:
# Extract linguistic features (Punctuation count and Sentiment score)
def extract_linguistic_features(text):
    punctuation_count = len(re.findall(r'[!?.]', text))
    sentiment_score = TextBlob(text).sentiment.polarity
    return [punctuation_count, sentiment_score]


X_linguistic = np.array([extract_linguistic_features(review) for review in dataset['review_text']])


In [ ]:
# Combine textual and linguistic features
X_combined = np.hstack((X_textual.toarray(), X_linguistic)

In [ ]:
# Split dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(X_combined, pseudo_labels, test_size=0.2, random_state=42)


In [ ]:
nb = MultinomialNB()
nb.fit(X_train, y_train)
nb_preds = nb.predict(X_test)
print("Naïve Bayes Accuracy:", accuracy_score(y_test, nb_preds))


In [ ]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
rf_preds = rf.predict(X_test)
print("Random Forest Accuracy:", accuracy_score(y_test, rf_preds))


In [ ]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_preds = lr.predict(X_test)
print("Logistic Regression Accuracy:", accuracy_score(y_test, lr_preds))


In [ ]:
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)
svm_preds = svm.predict(X_test)
print("SVM Accuracy:", accuracy_score(y_test, svm_preds))


In [ ]:
# Print classification reports
print("SVM:\n", classification_report(y_test, svm_preds))
print("Naïve Bayes:\n", classification_report(y_test, nb_preds))
print("Random Forest:\n", classification_report(y_test, rf_preds))
print("Logistic Regression:\n", classification_report(y_test, lr_preds))
